# Gradio

Gradio에 미리 구축된 컴포넌트를 이용해서 채팅 서비스 테스트

In [ ]:
!pip install gradio
!pip install openai

## 프롬프트 템플릿

In [2]:
import json
with open("../student_ai/template_text.json", "r", encoding="utf-8") as file:
    template_text = json.load(file)
template_text

{'chat_instruction': ['너는 지금부터 사용자에게 공부를 배우는 학생 ai 야,',
  '너는 초등학생, 중학생 정도의 어린 학생 ai 라서 사용자에게 존댓말만 써야 해.',
  '너는 사용자에게 공부를 배우고 사용자에게 질문함으로써 상호작용할 거야,',
  '질문은 한 번에 최대 2개씩만 해.',
  '너는 학생 ai 니까 사용자가 말한 내용 안에서만 질문하고 대답해.',
  '사용자가 질문해도, 학생 ai 이기 때문에 사용자에게 배운 내용이 아니면 모른다고 해야 돼.',
  '사용자가 틀린 얘기를 해도, 너는 아는 게 적은 학생 ai 니까 그 말이 맞다고 믿어야 돼.'],
 'eval_instruction': ['점수와 피드백 부분을 채워줘. 점수는 100점 만점으로 해줘.',
  '문제: 반복문이란 무엇인가?',
  '풀이: 반복문은 반복하는 명령문이다.',
  '답: 반복문이란 프로그램 내에서 똑같은 명령을 일정 횟수만큼 반복하여 수행하도록 제어하는 명령문입니다.',
  '점수와 피드백: 70:설명이 빈약합니다.',
  '문제: 샤이니의 멤버 구성은 어떻게 되는가?',
  '풀이: 샤이니는 온유, 정찬, 키, 인호 4명으로 이루어진 4인조 그룹입니다.',
  '답: 샤이니는 온유, 종현, 키, 민호, 태민 5명으로 이루어진 5인조 그룹입니다.',
  '점수와 피드백: 40:샤이니는 4인조 그룹이 아닌 온유, 종현, 키, 민호, 태민 5명으로 이루어진 5인조 그룹입니다.',
  '문제: {question}',
  '풀이: {test_paper}',
  '답: {answer}',
  '점수와 피드백: '],
 'test_instruction': ['너는 지금부터 사용자가 말해준 내용으로 시험을 보는 학생 ai 야.',
  '사용자가 말한 내용 안에서만 대답을 해.',
  "사용자가 언급하지 않은 내용이 시험 문제로 나오면 '모르겠어요'라고 대답해."]}

## 채팅 역할 제공 및 메시지 기록

In [3]:
def make_message(msg, history):
    messages = []

    for h in history:
        messages.append({"role": "user", "content": h[0]})
        messages.append({"role": "assistant", "content": h[1]})

    messages.append({"role": "system",
                     "content": '\n'.join(template_text['chat_instruction'])})

    messages.append({"role": "user", "content": msg})
    return messages

## Gradio 연결

In [ ]:
import gradio as gr
from openai import OpenAI

client = OpenAI()

def chat(msg, history):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=make_message(msg, history),
        temperature=0.5,
    )
    history.append((msg, response.choices[0].message.content))
    return "", history

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(chat, [msg, chatbot], [msg, chatbot])

demo.launch(share=True)

![Gradio](https://github.com/Aivle4th-team3/Student-AI-Package/assets/26417221/b28f5184-7644-45f1-a4b7-9ea316b8d014)